In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from fancyimpute import IterativeImputer

In [2]:
ecomonic_df=pd.read_csv('worldBankData\economicData.csv')

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\hp\AppData\Local\Temp\ipykernel_12420\2839501798.py:1: SyntaxWarning: invalid escape sequence '\e'
  ecomonic_df=pd.read_csv('worldBankData\economicData.csv')


In [23]:
df=ecomonic_df.copy()


In [24]:
import seaborn as sns
import matplotlib.pyplot as plt
encoder=LabelEncoder()
df['Country Name']=encoder.fit_transform(df['Country Name'])




In [25]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
df_not_to_scale=df[['Country Name','who_region','world_bank_income_level','Year']]
df=scaler.fit_transform(df.drop(['Country Name','who_region','world_bank_income_level','Year'],axis=1))
df=pd.DataFrame(df,columns=ecomonic_df.columns[4:])
df=pd.concat([df_not_to_scale,df],axis=1)

Filling Nulls for the columns "Adjusted net national income (annual % growth)"

In [ ]:
import pandas as pd
from fancyimpute import IterativeImputer

def impute_all_columns(df):
    columns_to_impute = df.columns[4:]  
    imputer = IterativeImputer()
    df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])
    return df

print("Before imputation:", df.shape)

df = impute_all_columns(df)

print("After imputation:", df.shape)


In [ ]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from joblib import Parallel, delayed

def impute_column(df, column, columns_to_use):

    X = df[columns_to_use].drop(columns=[column], errors='ignore')
    y = df[column]


    X = X[y.notna()]
    y = y.dropna()

 
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)  # Use multiple cores
    model.fit(X, y)


    missing_rows = df[column].isnull()
    df.loc[missing_rows, column] = model.predict(df.loc[missing_rows, columns_to_use].drop(columns=[column], errors='ignore'))

    return df

def process_file(file, base_dir):
    df = pd.read_csv(base_dir + file)

    columns_to_use = df.columns[4:]  
    columns_with_missing = [col for col in columns_to_use if df[col].isnull().any()]

    df = Parallel(n_jobs=-1)(delayed(impute_column)(df, column, columns_to_use) for column in columns_with_missing)

    df = df[-1]  
    df.to_csv(base_dir + 'imputed_' + file, index=False)
    print(f"Done processing {file}") 

base_dir = 'worldBankData/'
process_file('economicData.csv', base_dir)


KeyboardInterrupt: 

In [7]:
df=pd.read_csv('worldBankData\imputed_social.csv')
df.isnull().sum()

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\hp\AppData\Local\Temp\ipykernel_700\3456981994.py:1: SyntaxWarning: invalid escape sequence '\i'
  df=pd.read_csv('worldBankData\imputed_social.csv')


Country Name                                                                                        0
who_region                                                                                          0
world_bank_income_level                                                                             0
Year                                                                                                0
Women Business and the Law Index Score (scale 1-100)                                                0
                                                                                                   ..
People with basic handwashing facilities including soap and water, urban (% of urban population)    0
Permanent cropland (% of land area)                                                                 0
Individuals using the Internet (% of population)                                                    0
Individuals using the Internet, female (% of female population)                   